# Analyzing Geospatial Data

```{contents}
:local:
:depth: 2
```

## Introduction

## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

```bash
jupyter lab
```

In [ ]:
# %pip install pygis

In [ ]:
import ee
import geemap
geemap.set_proxy(33210)
geemap.ee_initialize()

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

## Earth Engine data reductions

### List reductions

In [ ]:
values = ee.List.sequence(1, 10)

print(values.getInfo())

In [ ]:
count = values.reduce(ee.Reducer.count())
print(count.getInfo())# 10

In [ ]:
min_value = values.reduce(ee.Reducer.min())
print(min_value.getInfo())  # 1

In [ ]:
max_value = values.reduce(ee.Reducer.max())
print(max_value.getInfo())  # 10

In [ ]:
min_max_value = values.reduce(ee.Reducer.minMax())
min_max_value.getInfo()

In [ ]:
mean_value = values.reduce(ee.Reducer.mean())
print(mean_value.getInfo())  # 5.5

In [ ]:
median_value = values.reduce(ee.Reducer.median())
print(median_value.getInfo())  # 5.5

In [ ]:
sum_value = values.reduce(ee.Reducer.sum())
print(sum_value.getInfo())  # 55

In [ ]:
std_value = values.reduce(ee.Reducer.stdDev())
print(std_value.getInfo())  # 2.8723

### ImageCollection reductions

In [ ]:
Map = geemap.Map()

# Load an image collection, filtered so it's not too much data.
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2021-01-01', '2021-12-31')
    .filter(ee.Filter.eq('WRS_PATH', 44))
    .filter(ee.Filter.eq('WRS_ROW', 34))
)

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.median())

# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B5_median', 'B4_median', 'B3_median'], 'gamma': 2}
Map.setCenter(-122.3355, 37.7924, 8)
Map.addLayer(median, vis_param)
Map

In [ ]:
median = collection.median()
print(median.bandNames().getInfo())

### Image reductions

In [ ]:
Map = geemap.Map()
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318').select(['B4', 'B3', 'B2'])
maxValue = image.reduce(ee.Reducer.max())
Map.centerObject(image, 8)
Map.addLayer(image, {}, 'Original image')
Map.addLayer(maxValue, {'max': 13000}, 'Maximum value image')
Map

### FeatureCollection reductions

In [ ]:
Map = geemap.Map()

# 将美国人口普查数据加载为 FeatureCollection。
census = ee.FeatureCollection('TIGER/2010/Blocks')

# 过滤集合以仅包括俄勒冈州本顿县。
benton = census.filter(
    ee.Filter.And(ee.Filter.eq('statefp10', '41'), ee.Filter.eq('countyfp10', '003'))
)

# 展示本顿县人口普查区。
Map.setCenter(-123.27, 44.57, 13)
Map.addLayer(benton)
Map


In [ ]:
# 计算指定属性的总和。
properties = ['pop10', 'housing10']
sums = benton.filter(ee.Filter.notNull(properties)).reduceColumns(
    **{'reducer': ee.Reducer.sum().repeat(2), 'selectors': properties}
)
#保留所有 pop10 和 housing10 都非空的记录。.repeat(2) 指定这个求和操作重复两次（因为有两个属性：pop10 和 housing10）。selectors: 指定要参与计算的列（属性）。这里是 ['pop10', 'housing10']。
# 打印结果字典。
print(sums.getInfo())


In [ ]:
#对特定属性直接计算
print(benton.aggregate_sum('pop10'))  # 85579
print(benton.aggregate_sum('housing10'))  # 36245

In [ ]:
benton.aggregate_stats('pop10')

## Image descriptive statistics

In [ ]:
Map = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(centroid).first()
vis = {'min': 0, 'max': 65455, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

Map.centerObject(centroid, 8)
Map.addLayer(image, vis, "Landsat-8")
Map

In [ ]:
#查看影像属性名称
image.propertyNames()

In [ ]:
#要查看属性值，用get
image.get('CLOUD_COVER')  # 0.05

In [ ]:
#字典形式获得影像所有的属性和值
props = geemap.image_props(image)
props

In [ ]:
#获得各波段的最大最小均值标准差、和
stats = geemap.image_stats(image, scale=30)
stats

## Zonal statistics with Earth Engine

### Zonal statistics计算区内均值，最大值

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NASA SRTM
dem = ee.Image('USGS/SRTMGL1_003')
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add 5-year Landsat TOA composite
landsat = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

# Add US Census States
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')
Map

In [ ]:
#根据矢量数据来分区计算各区均值，像元大小为1000*1000m
out_dem_stats = 'dem_stats.csv'
geemap.zonal_stats(
    dem, states, out_dem_stats, statistics_type='MEAN', scale=1000, return_fc=False
)

In [ ]:
out_landsat_stats = 'landsat_stats.csv'
geemap.zonal_stats(
    landsat,
    states,
    out_landsat_stats,
    statistics_type='MEAN',
    scale=1000,
    return_fc=False,
)

### Zonal statistics by group分区计算各类别和、百分比

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NLCD data
dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

# Add US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')

# Add NLCD legend
Map.add_legend(title='NLCD Land Cover', builtin_legend='NLCD')
Map

In [ ]:
nlcd_stats = 'nlcd_stats.csv'

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='SUM',
    denominator=1e6,#默认为1m，此处该为平方千米
    decimal_places=2,
)

In [ ]:
nlcd_stats = 'nlcd_stats_pct.csv'

geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='PERCENTAGE',
    denominator=1e6,
    decimal_places=2,
)

### Zonal statistics with two images 利用两个img分类计算，eg：根据用地类型来就算各类型蒸散发

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
dem = ee.Image('USGS/3DEP/10m')
vis = {'min': 0, 'max': 4000, 'palette': 'terrain'}
Map.addLayer(dem, vis, 'DEM')
Map

In [ ]:
landcover = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')
Map.add_legend(title='NLCD Land Cover Classification', builtin_legend='NLCD')

In [ ]:
stats = geemap.image_stats_by_zone(dem, landcover, reducer='MEAN')
stats

In [ ]:
stats.to_csv('mean.csv', index=False)

In [ ]:
geemap.image_stats_by_zone(dem, landcover, out_csv="std.csv", reducer='STD')

## Coordinate grids and fishnets

### Creating coordinate grids

In [ ]:
#经度网1
lat_grid = geemap.latitude_grid(step=5.0, west=-180, east=180, south=-85, north=85)

In [ ]:
Map = geemap.Map()
style = {'fillColor': '00000000'}
Map.addLayer(lat_grid.style(**style), {}, 'Latitude Grid')
Map

In [ ]:
#将数据转换为pandas dataframe
df = geemap.ee_to_df(lat_grid)
df

In [ ]:
#纬度网
lon_grid = geemap.longitude_grid(step=5.0, west=-180, east=180, south=-85, north=85)

In [ ]:
Map = geemap.Map()
style = {'fillColor': '00000000'}
Map.addLayer(lon_grid.style(**style), {}, 'Longitude Grid')
Map

In [ ]:
#经纬网
grid = geemap.latlon_grid(
    lat_step=10, lon_step=10, west=-180, east=180, south=-85, north=85
)

In [ ]:
Map = geemap.Map()
style = {'fillColor': '00000000'}
Map.addLayer(grid.style(**style), {}, 'Coordinate Grid')
Map

### Creating fishnets

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.user_roi

if roi is None:
    roi = ee.Geometry.BBox(-112.8089, 33.7306, -88.5951, 46.6244)
    Map.addLayer(roi, {}, 'ROI')
    Map.user_roi = None

Map.centerObject(roi)

In [ ]:
fishnet = geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=1)
style = {'color': 'blue', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = Map.user_roi

if roi is None:
    roi = ee.Geometry.Polygon(
        [
            [
                [-64.602356, -1.127399],
                [-68.821106, -12.625598],
                [-60.647278, -22.498601],
                [-47.815247, -21.111406],
                [-43.860168, -8.913564],
                [-54.582825, -0.775886],
                [-60.823059, 0.454555],
                [-64.602356, -1.127399],
            ]
        ]
    )
    Map.addLayer(roi, {}, 'ROI')

Map.centerObject(roi)
Map

In [ ]:
fishnet = geemap.fishnet(roi, rows=6, cols=8, delta=1)
style = {'color': 'blue', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')

## Extracting pixel values

### Extracting values to points根据点提取值

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2},
    'Landsat 7',
)
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 1)
Map

In [ ]:
in_shp = 'us_cities.shp'
url = 'https://github.com/giswqs/data/raw/main/us/us_cities.zip'
geemap.download_file(url)

In [ ]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'Cities')

In [ ]:
geemap.extract_values_to_points(in_fc, dem, out_fc="dem.shp")

In [ ]:
geemap.shp_to_gdf("dem.shp")

In [ ]:
geemap.extract_values_to_points(in_fc, landsat7, 'landsat.csv')

In [ ]:
geemap.csv_to_df('landsat.csv')

### Extracting pixel values along a transect

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("TERRAIN")

image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM DEM', True, 0.5)
Map

In [ ]:
line = Map.user_roi
if line is None:
    line = ee.Geometry.LineString(
        [[-120.2232, 36.3148], [-118.9269, 36.7121], [-117.2022, 36.7562]]
    )
    Map.addLayer(line, {}, "ROI")
Map.centerObject(line)

In [ ]:
reducer = 'mean'
transect = geemap.extract_transect(
    image, line, n_segments=100,#等距离分为多少个点
    reducer=reducer, to_pandas=True
)
transect

In [ ]:
geemap.line_chart(
    data=transect,
    x='distance',
    y='mean',
    markers=True,
    x_label='Distance (m)',
    y_label='Elevation (m)',
    height=400,
)

In [ ]:
transect.to_csv('transect.csv')

### Interactive region reduction

In [ ]:
Map = geemap.Map()

collection = (
    ee.ImageCollection('MODIS/061/MOD13A2')
    .filterDate('2015-01-01', '2019-12-31')
    .select('NDVI')
)

image = collection.toBands()

ndvi_vis = {
    'min': 0.0,
    'max': 9000.0,
    'palette': 'ndvi',
}

Map.addLayer(image, {}, 'MODIS NDVI Time-series')
Map.addLayer(image.select(0), ndvi_vis, 'First image')

Map

In [ ]:
dates = geemap.image_dates(collection).getInfo()
dates

In [ ]:
len(dates)

In [ ]:
Map.set_plot_options(add_marker_cluster=True)
Map.roi_reducer = ee.Reducer.mean()
Map

In [ ]:
Map.extract_values_to_points('ndvi.csv')

## Mapping available image count卫星在划定时间段内通过次数图

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
image = geemap.image_count(
    collection, region=None, start_date='2021-01-01', end_date='2022-01-01', clip=False
)

In [ ]:
Map = geemap.Map()
vis = {'min': 0, 'max': 60, 'palette': 'coolwarm'}
Map.addLayer(image, vis, 'Image Count')
Map.add_colorbar(vis, label='Landsat 8 Image Count')

countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
style = {"color": "00000088", "width": 1, "fillColor": "00000000"}
Map.addLayer(countries.style(**style), {}, "Countries")
Map

## Cloud-free composites

In [ ]:
Map = geemap.Map()

collection = ee.ImageCollection('LANDSAT/LC08/C02/T1').filterDate(
    '2021-01-01', '2022-01-01'
)

composite = ee.Algorithms.Landsat.simpleComposite(collection)

vis_params = {'bands': ['B5', 'B4', 'B3'], 'max': 128}

Map.setCenter(-122.3578, 37.7726, 10)
Map.addLayer(composite, vis_params, 'TOA composite')
Map


In [ ]:
customComposite = ee.Algorithms.Landsat.simpleComposite(
    **{'collection': collection, 'percentile': 30, 'cloudScoreRange': 5}
)

Map.addLayer(customComposite, vis_params, 'Custom TOA composite')
Map.setCenter(-105.4317, 52.5536, 11)

In [ ]:
vis_params = [
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 128},
    {'bands': ['B7', 'B6', 'B4'], 'min': 0, 'max': 128},
    {'bands': ['B6', 'B5', 'B2'], 'min': 0, 'max': 128},
]

labels = [
    'Natural Color (4, 3, 2)',
    'Color Infrared (5, 4, 3)',
    'Short-Wave Infrared (7, 6 4)',
    'Agriculture (6, 5, 2)',
]

In [ ]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    center=[37.7726, -122.1578],
    zoom=9,
    ee_objects=[composite],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

## Quality mosaicking

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
roi = countries.filter(ee.Filter.eq('ISO_A3', 'USA'))
Map.addLayer(roi, {}, 'roi')
Map

In [ ]:
start_date = '2020-01-01'
end_date = '2021-01-01'
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(roi)
    .filterDate(start_date, end_date)
)

In [ ]:
median = collection.median()
vis_rgb = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0,
    'max': 0.4,
}
Map.addLayer(median, vis_rgb, 'Median')
Map

In [ ]:
def add_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
def add_time(image):
    date = ee.Date(image.date())

    img_date = ee.Number.parse(date.format('YYYYMMdd'))
    image = image.addBands(ee.Image(img_date).rename('date').toInt())

    img_month = ee.Number.parse(date.format('M'))
    image = image.addBands(ee.Image(img_month).rename('month').toInt())

    img_doy = ee.Number.parse(date.format('D'))
    image = image.addBands(ee.Image(img_doy).rename('day').toInt())

    return image

In [ ]:
images = collection.map(add_ndvi).map(add_time)

In [ ]:
greenest = images.qualityMosaic('NDVI')

In [ ]:
greenest.bandNames()

In [ ]:
ndvi = greenest.select('NDVI')
vis_ndvi = {'min': 0, 'max': 1, 'palette': 'ndvi'}
Map.addLayer(ndvi, vis_ndvi, 'NDVI')
Map.add_colorbar(vis_ndvi, label='NDVI', layer_name='NDVI')
Map

In [ ]:
Map.addLayer(greenest, vis_rgb, 'Greenest pixel')

In [ ]:
vis_month = {'palette': ['red', 'blue'], 'min': 1, 'max': 12}
Map.addLayer(greenest.select('month'), vis_month, 'Greenest month')
Map.add_colorbar(vis_month, label='Month', layer_name='Greenest month')

In [ ]:
vis_doy = {'palette': ['brown', 'green'], 'min': 1, 'max': 365}
Map.addLayer(greenest.select('doy'), vis_doy, 'Greenest day')
Map.add_colorbar(vis_doy, label='Day of year', layer_name='Greenest day')

## Interactive charts

### Chart Overview

### Data table charts

In [ ]:
data = geemap.examples.get_path('countries.geojson')
df = geemap.geojson_to_df(data)
df.head()

In [ ]:
geemap.bar_chart(
    data=df,
    x='NAME',
    y='POP_EST',
    x_label='Country',
    y_label='Population',
    descending=True,
    max_rows=30,
    title='World Population',
    height=500,
    layout_args={'title_x': 0.5, 'title_y': 0.85},
)

In [ ]:
geemap.pie_chart(
    data=df,
    names='NAME',
    values='POP_EST',
    max_rows=30,
    height=600,
    title='World Population',
    legend_title='Country',
    layout_args={'title_x': 0.47, 'title_y': 0.87},
)

In [ ]:
data = geemap.examples.get_path('life_exp.csv')
df = geemap.csv_to_df(data)
df = df[df['continent'] == 'Oceania']
df.head()

In [ ]:
geemap.line_chart(
    df,
    x='year',
    y='lifeExp',
    color='country',
    x_label='Year',
    y_label='Life expectancy',
    legend_title='Country',
    height=400,
    markers=True,
)

### Earth Engine object charts

In [ ]:
import geemap.chart as chart

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
collection = ee.FeatureCollection('projects/google/charts_feature_example')
Map.addLayer(collection, {}, "Ecoregions")
Map

#### Chart by feature按要素（用地类型）对每一个要素单独作图

In [ ]:
#Collection to dataframe
features = collection.select('[0-9][0-9]_tmean|label')
#'[0-9][0-9]_tmean|label' 是一个正则表达式，[0-9][0-9]_tmean 匹配以两位数字开头并以 _tmean 结尾的波段名（例如 01_tmean, 12_tmean）。
#|label 表示“或”运算符，匹配名为 label 的波段。最终结果是筛选出所有名称符合这些规则的波段。
df = geemap.ee_to_df(features, sort_columns=True)
# sort_columns=Trueb保证以升序对列名排序
df

In [ ]:
xProperty = "label"
yProperties = df.columns[:12]

labels = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
]
colors = [
    '#604791',
    '#1d6b99',
    '#39a8a7',
    '#0f8755',
    '#76b349',
    '#f0af07',
    '#e37d05',
    '#cf513e',
    '#96356f',
    '#724173',
    '#9c4f97',
    '#696969',
]
title = "Average Monthly Temperature by Ecoregion"
xlabel = "Ecoregion"
ylabel = "Temperature"

In [ ]:
options = {
    "labels": labels,
    "colors": colors,
    "title": title,
    "x_label": xlabel,
    "y_label": ylabel,
    "legend_location": "top-left",
    "height": "500px",
}

In [ ]:
chart.feature_by_feature(features, xProperty, yProperties, **options)

#### Chart by property按属性（月）分别对每一个要素出图 

In [ ]:
features = collection.select('[0-9][0-9]_ppt|label')
df = geemap.ee_to_df(features, sort_columns=True)
df

In [ ]:
keys = df.columns[:12]
values = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
]
xProperties = dict(zip(keys, values))
seriesProperty = "label"

In [ ]:
options = {
    'title': "Average Ecoregion Precipitation by Month",
    'colors': ['#f0af07', '#0f8755', '#76b349'],
    'x_label': "Month",
    'y_label': "Precipitation (mm)",
    'legend_location': "top-left",
    "height": "500px",
}

In [ ]:
chart.feature_by_property(features, xProperties, seriesProperty, **options)

#### Feature histograms 直方图

In [ ]:
source = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').toBands()
region = ee.Geometry.Rectangle(-123.41, 40.43, -116.38, 45.14)
samples = source.sample(region, 5000)#随机生成点
prop = '07_ppt'

In [ ]:
options = {
    "title": 'July Precipitation Distribution for NW USA',
    "x_label": 'Precipitation (mm)',
    "y_label": 'Pixel count',
    "colors": ['#1d6b99'],
}

In [ ]:
chart.feature_histogram(samples, prop, **options)

In [ ]:
chart.feature_histogram(samples, prop, max_buckets=30, **options)

In [ ]:
chart.feature_histogram(samples, prop, min_bucket_width=0.5, **options)

In [ ]:
fig = chart.feature_histogram(samples, prop, min_bucket_width=3, max_buckets=30, **options)

## Unsupervised classification

In [ ]:
Map = geemap.Map()

point = ee.Geometry.Point([-88.0664, 41.9411])

image = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2022-01-01', '2022-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

region = image.geometry()
image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

Map.centerObject(region, 8)
Map.addLayer(image, vis_params, "Landsat-9")
Map

In [ ]:
geemap.get_info(image)

In [ ]:
image.get('DATE_ACQUIRED').getInfo()

In [ ]:
image.get('CLOUD_COVER').getInfo()

In [ ]:
training = image.sample(
    **{
        # "region": region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'Training samples')
Map

In [ ]:
geemap.ee_to_df(training.limit(5))

In [ ]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

In [ ]:
legend_dict = {
    'Forest': '#1c5f2c',
    'Developed, High Intensity': '#ab0000',
    'Cropland': '#ab6c28',
    'Open Water': '#466b9f',
    'Developed, Low Intensity': '#d99282'

}

palette = list(legend_dict.values())

Map.addLayer(
    result, {'min': 0, 'max': 4, 'palette': palette}, 'Labelled clusters'
)
Map.add_legend(title='Land Cover Type',legend_dict=legend_dict , position='bottomright')
Map

In [ ]:
geemap.download_ee_image(image, filename='unsupervised.tif', region=region, scale=90)

## Supervised classification

In [3]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2019-01-01', '2020-01-01')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
geemap.get_info(image)

Tree(nodes=(Node(name='Image  (7 bands)', nodes=(Node(icon='file', name='type: Image'), Node(name='bands: List…

In [5]:
image.get('DATE_ACQUIRED').getInfo()

'2019-09-24'

In [6]:
image.get('CLOUD_COVER').getInfo()

0.03

In [7]:
nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover').clip(image.geometry())
Map.addLayer(landcover, {}, 'NLCD Landcover')
Map

Map(bottom=25671.0, center=[37.60117623656667, -120.72875976562501], controls=(WidgetControl(options=['positio…

In [8]:
points = landcover.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,
    }
)
#训练样本数可能少于5000，因为无值像素会被舍弃
Map.addLayer(points, {}, 'training', False)

In [9]:
print(points.size().getInfo())

3575


In [10]:
#从指定的遥感图像 image 中，提取样本点 points 的波段特征值（SR_B1 到 SR_B7）。
#将样本点的地物类型标签 landcover 与波段特征关联，生成一个用于分类或回归任务的特征集合 features
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
label = 'landcover'
features = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

In [11]:
geemap.ee_to_df(features.limit(5))

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,landcover
0,0.024125,0.032815,0.064302,0.071975,0.293488,0.219568,0.111932,82
1,0.047060,0.059847,0.084103,0.088118,0.173835,0.137425,0.107532,24
2,0.035290,0.046318,0.077118,0.099502,0.249515,0.301050,0.188850,21
3,0.047308,0.065512,0.109513,0.146445,0.277840,0.302123,0.218550,82
4,0.058115,0.074918,0.104315,0.120375,0.197898,0.171882,0.138030,22


In [12]:
params = {

    'features': features,
    'classProperty': label,
    'inputProperties': bands,

}
classifier = ee.Classifier.smileCart(maxNodes=None).train(**params)

In [13]:
classified = image.select(bands).classify(classifier).rename('landcover')
Map.addLayer(classified.randomVisualizer(), {}, 'Classified')
Map

Map(bottom=25671.0, center=[37.60117623656667, -120.72875976562501], controls=(WidgetControl(options=['positio…

In [14]:
geemap.get_info(nlcd)

Tree(nodes=(Node(name='Image  (3 bands)', nodes=(Node(icon='file', name='type: Image'), Node(icon='file', name…

In [15]:
class_values = nlcd.get('landcover_class_values')
class_palette = nlcd.get('landcover_class_palette')
classified = classified.set({
    'landcover_class_values': class_values,
    'landcover_class_palette': class_palette
})

In [16]:
Map.addLayer(classified, {}, 'Land cover')
Map.add_legend(title="Land cover type", builtin_legend='NLCD')
Map

Map(bottom=25671.0, center=[37.60117623656667, -120.72875976562501], controls=(WidgetControl(options=['positio…

In [ ]:
geemap.download_ee_image(
    landcover,
    filename='supervised.tif',
    region=image.geometry(),
    scale=30
    )

## Accuracy assessment

In [17]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

img = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2021-01-01')
    .sort('CLOUDY_PIXEL_PERCENTAGE')#默认从云量最少的开始排序
    .first()
    .select('B.*')
)

vis_params = {'min': 100, 'max': 3500, 'bands': ['B11',  'B8',  'B3']}

Map.centerObject(point, 9)
Map.addLayer(img, vis_params, "Sentinel-2")
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
lc = ee.Image('ESA/WorldCover/v100/2020')
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remapValues = ee.List.sequence(0, 10)
label = 'lc'
lc = lc.remap(classValues, remapValues).rename(label).toByte()#将原有的 classValues 映射为 remapValues 中的对应值;将影像数据的标签改为 'lc';
#将影像数据的像素值类型转换为字节（Byte），即将像素值限制在 0 到 255 之间

In [19]:
sample = img.addBands(lc).stratifiedSample(**{#分层抽样
  'numPoints': 100,#每个类别采样100个点
  'classBand': label,#使用之前创建的类别标签 'lc' 作为分类字段
  'region': img.geometry(),
  'scale': 10,
  'geometries': True
})

In [20]:
sample = sample.randomColumn()#为样本集添加一个随机列（'random'），其值介于0和1之间
trainingSample = sample.filter('random <= 0.8')#将样本集分为训练集，包含随机值小于等于0.8的样本
validationSample = sample.filter('random > 0.8')#将样本集分为验证集，包含随机值大于0.8的样本。

In [21]:
sample

In [ ]:
#在 Google Earth Engine (GEE) 中训练一个 随机森林分类器 (smileRandomForest)。通过 trainingSample 作为训练数据，训练一个分类器以预测给定影像数据（img）中的类标签。
trainedClassifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(**{
  'features': trainingSample,# 训练样本集，包含输入特征和标签信息
  'classProperty': label,# 分类属性（目标变量），分类器将根据该属性来预测影像的分类
  'inputProperties': img.bandNames()# 输入特征（影像的波段名称），这些波段将用来训练模型
})

In [ ]:
#explain() 是一个方法，用于生成对分类器的解释或详细描述。
#它返回一个描述分类器内部结构的对象，通常包含有关分类器模型的各个方面的详细信息（例如，模型类型、特征、参数、训练过程中的决策规则等）。
print('Results of trained classifier', trainedClassifier.explain().getInfo())

In [ ]:
#计算并获取训练分类器的混淆矩阵，进而评估分类器的准确性。混淆矩阵输入类如果不是基于0或顺序，某些行列可能为空，因此上述remapValues该为从0开始序列
trainAccuracy = trainedClassifier.confusionMatrix()
trainAccuracy.getInfo()

In [ ]:
#计算并获取训练分类器的总体准确性，即正确分类的样本占总样本的比例（所有站点中正确映射的比例）
trainAccuracy.accuracy().getInfo()

In [ ]:
#计算Kappa系数，反映分类器分类结果的随机性。
trainAccuracy.kappa().getInfo()

In [ ]:
#计算了验证集的混淆矩阵
validationSample = validationSample.classify(trainedClassifier)#classify()用于使用已训练的分类器trainedClassifier对验证样本validationSample进行分类。
#结果是，validationSample 会包含两个属性：label：真实的标签（实际类别）；classification：由分类器预测的标签
validationAccuracy = validationSample.errorMatrix(label, 'classification')#改方法计算错误矩阵（也称为 混淆矩阵），用来比较真实标签（label）和分类结果（classification）的差异，评估分类器的性能。
validationAccuracy.getInfo()

In [ ]:
#计算并获取验证集的整体准确度，它通过错误矩阵来计算。=对角线元素/所有元素
validationAccuracy.accuracy().getInfo()

In [ ]:
#算生产者准确率，这个指标衡量的是 每个类别 被正确分类的比例。它反映了对每个类别的 真实样本 被分类器正确识别的能力。
#在该处为分类地图正确反映特定土地覆盖类型的概率
validationAccuracy.producersAccuracy().getInfo()

In [ ]:
#消费者准确率，它衡量的是每个类别被 正确分类 的比例，反映了分类器对每个类别的 误分类情况，即 每个类别预测为其他类别的概率。
#此处为地图上类出现在地面上的概率
validationAccuracy.consumersAccuracy().getInfo()

In [ ]:
import csv

with open("training.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainAccuracy.getInfo())

with open("validation.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(validationAccuracy.getInfo())

In [ ]:

imgClassified = img.classify(trainedClassifier)

In [ ]:
classVis = {
  'min': 0,
  'max': 10,
  'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4',
            'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
}
Map.addLayer(lc, classVis, 'ESA Land Cover', False)
Map.addLayer(imgClassified, classVis, 'Classified')
Map.addLayer(trainingSample, {'color': 'black'}, 'Training sample')
Map.addLayer(validationSample, {'color': 'white'}, 'Validation sample')
Map.add_legend(title='Land Cover Type', builtin_legend='ESA_WorldCover')
Map.centerObject(img)
Map

## Using locally trained machine learning models

In [22]:
import pandas as pd
from geemap import ml
from sklearn import ensemble

### Train a model locally using scikit-learn

In [23]:
url = "https://raw.githubusercontent.com/gee-community/geemap/master/examples/data/rf_example.csv"
df = pd.read_csv(url)
df

,B2,B3,B4,B5,B6,B7,landcover
0,0.139846,0.114738,0.109982,0.119542,0.125795,0.105720,0
1,0.130316,0.109207,0.107499,0.140210,0.132006,0.108497,0
2,0.146690,0.135766,0.146550,0.225686,0.218105,0.167111,0
3,0.119413,0.108924,0.105196,0.144868,0.159775,0.122056,0
4,0.155492,0.139932,0.137486,0.151377,0.153771,0.133134,0
...,...,...,...,...,...,...,...
93,0.117331,0.092176,0.062548,0.020362,0.005813,0.004047,2
94,0.118353,0.093785,0.060253,0.020083,0.007317,0.004719,2
95,0.123362,0.095831,0.069663,0.027320,0.011386,0.008357,2
96,0.122907,0.100083,0.079527,0.024564,0.008570,0.006321,2


In [24]:
feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = "landcover"

In [25]:
X = df[feature_names]
y = df[label]
n_trees = 10
rf = ensemble.RandomForestClassifier(n_trees).fit(X, y)#从 scikit-learn 的 ensemble 模块导入随机森林分类器，对模型进行训练。X 是特征数据（输入）。y 是目标标签（输出）。
# 随机森林会在 X 和 y 的基础上，构造 n_trees 棵决策树。
# 每棵树使用训练数据的随机子集（通过 Bootstrap抽样）。
# 每棵树在随机选择的特征子集上分裂节点。

### Convert a sklearn classifier object to a list of strings

In [26]:
#earth engine 不能直接使用scikit-learn模型，需转为字符串
trees = ml.rf_to_strings(rf, feature_names)

In [27]:
print(len(trees))

10


In [28]:
print(trees[0])

1) root 62 9999 9999 (1.6019873085288538)
  2) B7 <= 0.018568 16 0.0000 2 *
  3) B7 > 0.018568 62 0.6522 0
    6) B4 <= 0.083411 16 0.0000 1 *
    7) B4 > 0.083411 46 0.4861 0
      14) B2 <= 0.123464 30 0.2449 0
        28) B6 <= 0.187960 1 0.0000 0 *
        29) B6 > 0.187960 5 0.0000 1 *
      15) B2 > 0.123464 24 0.0000 0 *



### Convert sklearn classifier to GEE classifier

In [29]:
#将转换的字符串转为gee分类器
ee_classifier = ml.strings_to_classifier(trees)
ee_classifier.getInfo()

{'type': 'Classifier.decisionTreeEnsemble',
 'treeStrings': ['1) root 62 9999 9999 (1.6019873085288538)\n  2) B7 <= 0.018568 16 0.0000 2 *\n  3) B7 > 0.018568 62 0.6522 0\n    6) B4 <= 0.083411 16 0.0000 1 *\n    7) B4 > 0.083411 46 0.4861 0\n      14) B2 <= 0.123464 30 0.2449 0\n        28) B6 <= 0.187960 1 0.0000 0 *\n        29) B6 > 0.187960 5 0.0000 1 *\n      15) B2 > 0.123464 24 0.0000 0 *\n',
  '1) root 63 9999 9999 (1.3242134717002707)\n  2) B3 <= 0.100830 63 0.6647 2\n    4) B5 <= 0.085596 25 0.0000 2 *\n    5) B5 > 0.085596 18 0.0000 1 *\n  3) B3 > 0.100830 63 0.6647 2\n    6) B2 <= 0.116759 1 0.0000 1 *\n    7) B2 > 0.116759 19 0.0000 0 *\n',
  '1) root 62 9999 9999 (1.7785888007069888)\n  2) B3 <= 0.107344 62 0.6597 1\n    4) B6 <= 0.047678 24 0.0000 2 *\n    5) B6 > 0.047678 46 0.5262 2\n      10) B3 <= 0.095034 16 0.0000 1 *\n      11) B3 > 0.095034 22 0.1049 1\n        22) B7 <= 0.112588 2 0.0000 0 *\n        23) B7 > 0.112588 4 0.0000 1 *\n  3) B3 > 0.107344 62 0.6597 

### Classify image using GEE classifier

In [40]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery).
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
image = ee.Algorithms.Landsat.simpleComposite(
    collection=l8.filterDate('2018-01-01', '2018-12-31'), asFloat=True
)

In [44]:
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
Map = geemap.Map()
Map.addLayer(l8,{},'l8')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

EEException: Landsat.simpleComposite: ImageCollection band names do not match a Landsat instrument.

In [42]:
classified = image.select(feature_names).classify(ee_classifier)

In [43]:
Map = geemap.Map(center=(37.75, -122.25), zoom=11)

# Map.addLayer(
#     image,
#     {"bands": ['B7', 'B5', 'B3'], "min": 0.05, "max": 0.55, "gamma": 1.5},
#     'image',
# )
Map.addLayer(
    classified,
    {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},
    'classification',
)
Map

EEException: Landsat.simpleComposite: ImageCollection band names do not match a Landsat instrument.

### Save trees to the cloud

In [ ]:
user_id = geemap.ee_user_id()
asset_id = user_id + "/random_forest_strings_test"
asset_id

In [ ]:
ml.export_trees_to_fc(trees, asset_id)

In [ ]:
rf_fc = ee.FeatureCollection(asset_id)
another_classifier = ml.fc_to_classifier(rf_fc)
classified = image.select(feature_names).classify(another_classifier)

### Save trees locally

In [ ]:
out_csv = "trees.csv"
ml.trees_to_csv(trees, out_csv)
another_classifier = ml.csv_to_classifier(out_csv)
classified = image.select(feature_names).classify(another_classifier)

## Sankey diagrams 绘制类型多年变化图

In [ ]:
import sankee

sankee.datasets.LCMS_LC.sankify(
    years=[1990, 2000, 2010, 2020],
    region=ee.Geometry.Point([-122.192688, 46.25917]).buffer(2000),
    max_classes=3,
    title="Mount St. Helens Recovery",
)

In [ ]:
Map = geemap.Map(height=650)
Map

## Summary

整合计算:
1 List.reduce(ee.Reducer.min())将返回ComputedObject类型数值，List.reduce(ee.Reducer.minMax())将返回{‘min’： ，‘max’： }
2 对imageCollection使用 Collection.reduce(ee.Reducer.max()) collection.max()均可,输出为单幅图像 
3 对image用 image.reduce(ee.Reducer.max())，输出为一幅图像的波段整合

区域计算:
1 利用shp文件对单一信息img文件分区，要计算分区信息，利用geemap.zonal_stats()可以对单或者多波段数据，分区计算各区域内均值、最大值等数据，并导出为csv、shp等文件
2 利用shp文件对多类信息img文件分区，计算分区各类信息，利用geemap.zonal_stats_by_groups(),返回区内各类总面积或者面积占比
3 如果分区为img文件，则利用geemap.image_stats_by_zone()来计算各区最大、最小、综合、均值等，返回为Pandas Dataframe类型表格,可以利用result.to_csv()返回保存为csv文件

经纬网和渔网:
1 利用grid = geemap.latlon_grid(lat_step=10, lon_step=10, west=-180, east=180, south=-85, north=85)可以为地图添加经纬网，并设置经纬线步长，最大最小值。
2 如果要对绘制的roi进行网格分区，可以用geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=1)来进行分割分区。
3 本节分区可以用于上一节划分区域计算

提取像素值:
1 利用点提取值，利用geemap.extract_values_to_points()提取对应点的值，单多波段均可，输出可以为csv，shp等格式。
2 延切线提取像素值，利用transect = geemap.extract_transect()返回为Pandas Dataframe格式，接下来可以利用geemap.line_chart()绘制剖面曲线，
或者transect.to_csv()保存文件
3 交互式区域整合，可以利用ipyleaflet中的工具对一个数据集的点绘制序列曲线。或者利用Map.set_plot_options(add_marker_cluster=True)和
Map.roi_reducer = ee.Reducer.mean()来在地图选择所需点（reducer可更改，默认均值），最后通过Map.extract_values_to_points()来导出点数据为csv或者shp

对卫星通过划定时段次数计数:
1 利用geemap.image_count(collection, region=None, start_date=, end_date=, clip=False)对区域卫星通过次数计数

去云：
1 这一节landsat新数据无法用该方法，放弃。
2 利用geemap.linked_maps()可以生成n*m的地图，方便对比，

高质量合成：
1 Collection.qualityMosaic('NDVI')，对imageCollection对象中一个指定的质量波段的最大值选择，选择 NDVI 值最大的图像来生成该位置的像素，
返回生成一副imageCollection中根据相同位置像素中NDVI的一副图的像素最得到的单幅图像。

利用geemap创建图表：
data通常为csv和pandas Dataframe类型
1 柱状图：geemap.bar_chart()
2 饼图：geemap.pie_chart()
3 折线图：geemap.line_chart()

创建ee对象图表：
1 根据要素（图例类型）来创建图表：chart.feature_by_feature(features, xProperty, yProperties, **options)
2 根据属性（月份）创建图表：chart.feature_by_property(features, xProperties, seriesProperty, **options)
3 根据特征绘制直方图：chart.feature_histogram(samples, prop, **options)也可自定义直方图宽度，通过更改min_bucket_width, max_buckets参数

无监督分类：
需要根据训练处result观察，对每一类进行辨别，按顺序找到不同类型。

监督分类：
本节利用（CART）分类和回归树模型进行监督分类。
1 首先根据目标图像范围创立若干采样点数
2 利用ee.Classifier.smileCart(maxNodes=None).train(**params)初始化CART训练器并进行分类
3 利用image.select(bands).classify(classifier).rename('landcover')对Landsat进行分类生成分类图像
4 设置与land_cover图像相同的调色板放如图中，方便对比

利用随机森林方法对landcover进行分类并做准确性评估：
1 加载要分类图像img以及landcover图，将lnadcover值改为以开始的序列，避免混淆矩阵出现行或列为0的情况
2 将landcover波段加载到img，并基于img分层抽样生成若干采样点，并分为训练样本（80%）和检验样本（20%）
3 利用trainedClassifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train()训练随机森林分类器
4 得到训练样本混淆矩阵trainAccuracy = trainedClassifier.confusionMatrix()、整体准确度trainAccuracy.accuracy().getInfo()，Kappa系数trainAccuracy.kappa().getInfo()
5 4操作同样对检验样本得到混淆矩阵、整体准确度和Kappa系数
6 如果准确度可以接受，img.classify(trainedClassifier)，利用训练好的分类器对img进行分类